In [2]:
# test_email_model.ipynb
import sys
import os 
import pandas as pd

module_path = os.path.abspath(os.path.join(os.getcwd(), "../email_prioritizer"))
sys.path.append(module_path)

from email_prioritizer import EmailPrioritizer
from sklearn.model_selection import train_test_split

In [4]:
df_emails_data = pd.read_csv("../data/email_classification_dataset.csv")
df_emails_data["Label"] = df_emails_data['label'] == 'spam'
df_emails_data

,id,email,label,Label
0,2685,From: support@legitcompany.com\nSubject: Regar...,ham,False
1,5857,From: noreply@softwareupdates.com\nSubject: We...,ham,False
2,2399,From: noreply@softwareupdates.com\nSubject: Im...,ham,False
3,3244,From: info@customerservice.co\nSubject: Team S...,ham,False
4,2844,From: info@customerservice.co\nSubject: Team S...,ham,False
...,...,...,...,...
9995,6397,From: noreply@softwareupdates.com\nSubject: Ca...,ham,False
9996,7470,From: family@homemail.net\nSubject: Weekly New...,ham,False
9997,9273,From: team@projectmanagement.com\nSubject: Fee...,ham,False
9998,3192,From: accounts@billingcorp.com\nSubject: Photo...,ham,False


In [5]:
num_holdout = 100
df_emails_train, df_emails_holdout = train_test_split(df_emails_data, test_size=(num_holdout*1./df_emails_data.shape[0]), random_state=42)
df_emails_train

,id,email,label,Label
764,175,From: noreply@winner-prize.net\nSubject: Exclu...,spam,True
5019,7411,From: newsletter@info.org\nSubject: Regarding ...,ham,False
4903,9541,From: survey@retailfeedback.com\nSubject: Rega...,ham,False
8093,2428,From: newsletter@info.org\nSubject: Meeting Re...,ham,False
6843,8676,From: friend@personalmail.net\nSubject: Team S...,ham,False
...,...,...,...,...
5734,8128,From: john.doe@example.com\nSubject: Feedback ...,ham,False
5191,3891,From: noreply@softwareupdates.com\nSubject: Fe...,ham,False
5390,3096,From: friend@personalmail.net\nSubject: Regard...,ham,False
860,5645,From: noreply@softwareupdates.com\nSubject: In...,ham,False


In [49]:
x_col = 'email'
y_col = 'Label'
X = df_emails_train[x_col]
y = df_emails_train[y_col]
X_holdout = df_emails_holdout[x_col]
y_holdout = df_emails_holdout[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=42)

In [50]:
detector = EmailPrioritizer(model_type="logistic")
detector.fit(X_train, y_train, cv=3)
detector.evaluate(X_test, y_test)
df_results = pd.DataFrame({
    'email': X_holdout,
    'prediction': detector.predict(X_holdout),
    'true_label': y_holdout,
    'correct': detector.predict(X_holdout) == y_holdout,
    'score': detector.predict_proba(X_holdout)[:,1],
})
df_results

Running GridSearchCV for logistic...
Fitting 3 folds for each of 4 candidates, totalling 12 fits


/Users/joshsherfey/.pyenv/versions/3.12.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/joshsherfey/.pyenv/versions/3.12.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/joshsherfey/.pyenv/versions/3.12.12/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025


Best cross-val accuracy: 0.966
Best parameters: {'model__C': 10}

Evaluation Report:
              precision    recall  f1-score   support

       False       0.94      1.00      0.97      8328
        True       1.00      0.64      0.78      1473

    accuracy                           0.95      9801
   macro avg       0.97      0.82      0.87      9801
weighted avg       0.95      0.95      0.94      9801

Accuracy: 0.9456177941026426


,email,prediction,true_label,correct,score
6252,From: friend@personalmail.net\nSubject: Catchi...,False,False,True,0.005184
4684,From: noreply@softwareupdates.com\nSubject: Me...,False,False,True,0.003892
1731,From: friend@personalmail.net\nSubject: Photos...,False,False,True,0.003076
4742,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.906328
4521,From: team@projectmanagement.com\nSubject: Mee...,False,False,True,0.007133
...,...,...,...,...,...
3787,From: john.doe@example.com\nSubject: Regarding...,False,False,True,0.006839
9189,From: accounts@billingcorp.com\nSubject: Regar...,False,False,True,0.005554
7825,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.963681
7539,From: family@homemail.net\nSubject: Invoice fo...,False,False,True,0.016463


In [38]:
detector_rf = EmailPrioritizer(model_type="random_forest")
detector_rf.fit(X_train, y_train, cv=3)
detector_rf.evaluate(X_test, y_test)
df_results_rf = pd.DataFrame({
    'email': X_holdout,
    'prediction': detector.predict(X_holdout),
    'true_label': y_holdout,
    'correct': detector.predict(X_holdout) == y_holdout,
    'score': detector.predict_proba(X_holdout)[:,1],
})
df_results_rf

Running GridSearchCV for random_forest...
Fitting 3 folds for each of 12 candidates, totalling 36 fits

Best cross-val accuracy: 1.000
Best parameters: {'model__max_depth': None, 'model__min_samples_split': 2, 'model__n_estimators': 100}

Evaluation Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      2515
        True       1.00      1.00      1.00       455

    accuracy                           1.00      2970
   macro avg       1.00      1.00      1.00      2970
weighted avg       1.00      1.00      1.00      2970

Accuracy: 1.0


,email,prediction,true_label,correct,score
6252,From: friend@personalmail.net\nSubject: Catchi...,False,False,True,0.055840
4684,From: noreply@softwareupdates.com\nSubject: Me...,False,False,True,0.053313
1731,From: friend@personalmail.net\nSubject: Photos...,False,False,True,0.049915
4742,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.739259
4521,From: team@projectmanagement.com\nSubject: Mee...,False,False,True,0.050229
...,...,...,...,...,...
3787,From: john.doe@example.com\nSubject: Regarding...,False,False,True,0.042184
9189,From: accounts@billingcorp.com\nSubject: Regar...,False,False,True,0.042463
7825,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.694179
7539,From: family@homemail.net\nSubject: Invoice fo...,False,False,True,0.059157


In [39]:
detector_cb = EmailPrioritizer(model_type="catboost")
detector_cb.fit(X_train, y_train, cv=3)
detector_cb.evaluate(X_test, y_test)
df_results_cb = pd.DataFrame({
    'email': X_holdout,
    'prediction': detector.predict(X_holdout),
    'true_label': y_holdout,
    'correct': detector.predict(X_holdout) == y_holdout,
    'score': detector.predict_proba(X_holdout)[:,1],
})
df_results_cb

Running GridSearchCV for catboost...
Fitting 3 folds for each of 8 candidates, totalling 24 fits

Best cross-val accuracy: 1.000
Best parameters: {'model__depth': 4, 'model__iterations': 200, 'model__learning_rate': 0.05}

Evaluation Report:
              precision    recall  f1-score   support

       False       1.00      1.00      1.00      2515
        True       1.00      1.00      1.00       455

    accuracy                           1.00      2970
   macro avg       1.00      1.00      1.00      2970
weighted avg       1.00      1.00      1.00      2970

Accuracy: 1.0


,email,prediction,true_label,correct,score
6252,From: friend@personalmail.net\nSubject: Catchi...,False,False,True,0.055840
4684,From: noreply@softwareupdates.com\nSubject: Me...,False,False,True,0.053313
1731,From: friend@personalmail.net\nSubject: Photos...,False,False,True,0.049915
4742,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.739259
4521,From: team@projectmanagement.com\nSubject: Mee...,False,False,True,0.050229
...,...,...,...,...,...
3787,From: john.doe@example.com\nSubject: Regarding...,False,False,True,0.042184
9189,From: accounts@billingcorp.com\nSubject: Regar...,False,False,True,0.042463
7825,From: deals@best-offers.xyz\nSubject: Verify Y...,True,True,True,0.694179
7539,From: family@homemail.net\nSubject: Invoice fo...,False,False,True,0.059157


In [40]:
df_emails_data = pd.read_csv("../data/emails.csv")
df_emails_data

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
...,...,...
517396,zufferli-j/sent_items/95.,Message-ID: <26807948.1075842029936.JavaMail.e...
517397,zufferli-j/sent_items/96.,Message-ID: <25835861.1075842029959.JavaMail.e...
517398,zufferli-j/sent_items/97.,Message-ID: <28979867.1075842029988.JavaMail.e...
517399,zufferli-j/sent_items/98.,Message-ID: <22052556.1075842030013.JavaMail.e...


In [ ]:
df_emails_data_random = df_emails_data.sample(n=1000)
df_emails_data_random['email'] = df_emails_data_random['message']


,file,message
98863,ermis-f/inbox/169.,Message-ID: <30559528.1075855169455.JavaMail.e...
97422,dorland-c/sent_items/339.,Message-ID: <19680659.1075840418964.JavaMail.e...
448345,solberg-g/deleted_items/19.,Message-ID: <11771876.1075841563440.JavaMail.e...
54646,cuilla-m/deleted_items/188.,Message-ID: <12518766.1075853090439.JavaMail.e...
45510,campbell-l/notes_inbox/605.,Message-ID: <29528344.1075857924287.JavaMail.e...
...,...,...
311648,mann-k/discussion_threads/1786.,Message-ID: <14061033.1075845796018.JavaMail.e...
231068,kean-s/all_documents/6392.,Message-ID: <30988171.1075847650719.JavaMail.e...
257975,kitchen-l/_americas/gas/21.,Message-ID: <8668663.1075840848630.JavaMail.ev...
478807,taylor-m/archive/november1999/45.,Message-ID: <15334805.1075859980268.JavaMail.e...


In [45]:
import pandas as pd
import re

text = """Message-ID: <17189699.1075863688308.JavaMail.evans@thyme>
Date: Fri, 14 Jul 2000 06:59:00 -0700 (PDT)
From: phillip.allen@enron.com
To: joyce.teixeira@enron.com
Subject: Re: PRC review - phone calls
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Joyce Teixeira
X-cc: 
X-bcc: 
X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

any morning between 10 and 11:30"""

pattern = (
    r"Message-ID:\s*<(?P<MessageID>[^>]+)>\s*"
    r"Date:\s*(?P<Date>.*?)\s*"
    r"From:\s*(?P<From>.*?)\s*"
    r"To:\s*(?P<To>.*?)\s*"
    r"Subject:\s*(?P<Subject>.*?)\s*"
    r"Mime-Version:\s*(?P<MimeVersion>.*?)\s*"
    r"Content-Type:\s*(?P<ContentType>[^;]+);\s*charset=(?P<Charset>\S+)\s*"
    r"Content-Transfer-Encoding:\s*(?P<Encoding>.*?)\s*"
    r"X-From:\s*(?P<XFrom>.*?)\s*"
    r"X-To:\s*(?P<XTo>.*?)\s*"
    r"X-cc:\s*(?P<Xcc>.*?)\s*"
    r"X-bcc:\s*(?P<Xbcc>.*?)\s*"
    r"X-Folder:\s*(?P<XFolder>.*?)\s*"
    r"X-Origin:\s*(?P<XOrigin>.*?)\s*"
    r"X-FileName:\s*(?P<XFileName>.*?)\s*"
    r"(?P<Body>[\s\S]*)"
)

s = pd.Series([text])
df = s.str.extract(pattern, flags=re.DOTALL)
print(df.T)


                                                           0
MessageID        17189699.1075863688308.JavaMail.evans@thyme
Date                   Fri, 14 Jul 2000 06:59:00 -0700 (PDT)
From                                 phillip.allen@enron.com
To                                  joyce.teixeira@enron.com
Subject                         Re: PRC review - phone calls
MimeVersion                                              1.0
ContentType                                       text/plain
Charset                                             us-ascii
Encoding                                                7bit
XFrom                                        Phillip K Allen
XTo                                           Joyce Teixeira
Xcc                                                         
Xbcc                                                        
XFolder      \Phillip_Allen_Dec2000\Notes Folders\'sent mail
XOrigin                                              Allen-P
XFileName               

In [47]:
df_parsed_emails = df_emails_data_random.message.str.extract(pattern, flags=re.DOTALL)
df_parsed_emails

,MessageID,Date,From,To,Subject,MimeVersion,ContentType,Charset,Encoding,XFrom,XTo,Xcc,Xbcc,XFolder,XOrigin,XFileName,Body
98863,30559528.1075855169455.JavaMail.evans@thyme,"Tue, 18 Dec 2001 15:54:14 -0800 (PST)",noreply@ccomad3.uu.commissioner.com,frank.ermis@enron.com,Put up $5 to win $1 Million!,1.0,text/plain,ANSI_X3.4-1968,7bit,CBS SportsLine.com <noreply@ccomad3.uu.commiss...,"Ermis, Frank </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",,,"\Frank_Ermis_Jan2002_1\Ermis, Frank\Inbox",Ermis-F,,fermis (Non-Privileged).pst\n\n\t\n\t\n \t CBS...
97422,19680659.1075840418964.JavaMail.evans@thyme,"Mon, 10 Sep 2001 07:41:40 -0700 (PDT)",chris.dorland@enron.com,carlos.torres@enron.com,RE: Position and P&L,1.0,text/plain,us-ascii,7bit,"Dorland, Chris </O=ENRON/OU=NA/CN=RECIPIENTS/C...","Torres, Carlos </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,,"\ExMerge - Dorland, Chris\Sent Items",DORLAND-C,,chris dorland 6-26-02.PST\n\nOct Sumas trades\...
448345,11771876.1075841563440.JavaMail.evans@thyme,"Sun, 3 Feb 2002 07:37:01 -0800 (PST)",pete.davis@enron.com,pete.davis@enron.com,Start Date: 2/3/02; HourAhead hour: 10;\nCc: a...,1.0,text/plain,us-ascii,"7bit\nBcc: albert.meyers@enron.com, bill.willi...",Schedule Crawler<pete.davis@enron.com>@ENRON,"Davis, Pete </O=ENRON/OU=NA/CN=RECIPIENTS/CN=P...","Meyers, Albert </O=ENRON/OU=NA/CN=RECIPIENTS/C...",,"\ExMerge - Solberg, Geir\Deleted Items",SOLBERG-G,,geir solberg 6-26-02.PST\n\n\n\nStart Date: 2/...
54646,12518766.1075853090439.JavaMail.evans@thyme,"Thu, 25 Oct 2001 07:56:32 -0700 (PDT)",bryant@cheatsheets.net,cheatsheets@yahoogroups.com,[Footballguys] Update # 138 10-25 Week 7 Chea...,1.0,text/plain,us-ascii,7bit,Joe Bryant <bryant@cheatsheets.net>,Cheatsheets Mailing Yahoo <cheatsheets@yahoogr...,,,"\MCUILLA (Non-Privileged)\Cuilla, Martin\Delet...",Cuilla-M,,"MCUILLA (Non-Privileged).pst\n\nHi Folks,\n\nA..."
45510,29528344.1075857924287.JavaMail.evans@thyme,"Sun, 13 May 2001 12:33:00 -0700 (PDT)",enron.announcements@enron.com,all.houston@enron.com,Remote control your air-conditioner and heater,1.0,text/plain,us-ascii,7bit,Enron Announcements,All Enron Houston,,,\Larry_Campbell_Jun2001\Notes Folders\Notes inbox,Campbell-L,,lcampbel.nsf\n\nNewPower to offer another pilo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311648,14061033.1075845796018.JavaMail.evans@thyme,"Mon, 11 Dec 2000 06:17:00 -0800 (PST)",kay.mann@enron.com,pthompson@akllp.com,GE ContractPayment Summary,1.0,text/plain,us-ascii,7bit,Kay Mann,pthompson@akllp.com,,,\Kay_Mann_June2001_2\Notes Folders\Discussion ...,MANN-K,,"kmann.nsf\n\nThe latest, greatest...\n--------..."
231068,30988171.1075847650719.JavaMail.evans@thyme,"Fri, 5 Jan 2001 05:56:00 -0800 (PST)",mark.schroeder@enron.com,"steven.kean@enron.com, richard.shapiro@enron.c...",Benefits of competition in electricity\nCc: jo...,1.0,text/plain,us-ascii,"7bit\nBcc: john.sherriff@enron.com, joseph.hir...",Mark Schroeder,"Steven J Kean, Richard Shapiro, James D Steffe...","John Sherriff, Joseph P Hirl, Paul Mead",,\Steven_Kean_June2001_1\Notes Folders\All docu...,KEAN-S,,skean.nsf\n\nAs we all deal with the fallout f...
257975,8668663.1075840848630.JavaMail.evans@thyme,"Fri, 2 Nov 2001 08:01:04 -0800 (PST)",laura.luce@enron.com,louise.kitchen@enron.com,NY Event w/PEC & Keyspan\nCc: tammie.schoppe@e...,1.0,text/plain,us-ascii,7bit\nBcc: tammie.schoppe@enron.com,"Luce, Laura </O=ENRON/OU=NA/CN=RECIPIENTS/CN=L...","Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Schoppe, Tammie </O=ENRON/OU=NA/CN=RECIPIENTS/...",,"\ExMerge - Kitchen, Louise\'Americas\Gas",KITCHEN-L,,"louise kitchen 2-7-02.pst\n\nLouise,\nIt is my..."
478807,15334805.1075859980268.JavaMail.evans@thyme,"Mon, 22 Nov 1999 08:45:00 -0800 (PST)",fishkinc@hotmail.com,"mark.haedicke@enron.com, mark.taylor@enron.com...",,1.0,text/plain,us-ascii,7bit,"""CHARLES FISHKIN"" <fishkinc@hotmail.com>","Mark E Haedicke, Mark Taylor, Rick Buy, Tana J...",,,\Mark_Taylor _Dec_2000\Notes Folders\Archive\N...,Ta

In [48]:
df_results_new = pd.DataFrame({
    'email': df_emails_data_random['message'],
    'sender': df_parsed_emails['From'],
    'subject': df_parsed_emails['Subject'],
    'body': df_parsed_emails['Body'],
    'prediction': detector.predict(df_emails_data_random['message']),
    'score': detector.predict_proba(df_emails_data_random['message'])[:,1],
})
df_results_new

,email,sender,subject,body,prediction,score
98863,Message-ID: <30559528.1075855169455.JavaMail.e...,noreply@ccomad3.uu.commissioner.com,Put up $5 to win $1 Million!,fermis (Non-Privileged).pst\n\n\t\n\t\n \t CBS...,False,0.239466
97422,Message-ID: <19680659.1075840418964.JavaMail.e...,chris.dorland@enron.com,RE: Position and P&L,chris dorland 6-26-02.PST\n\nOct Sumas trades\...,False,0.154439
448345,Message-ID: <11771876.1075841563440.JavaMail.e...,pete.davis@enron.com,Start Date: 2/3/02; HourAhead hour: 10;\nCc: a...,geir solberg 6-26-02.PST\n\n\n\nStart Date: 2/...,False,0.163893
54646,Message-ID: <12518766.1075853090439.JavaMail.e...,bryant@cheatsheets.net,[Footballguys] Update # 138 10-25 Week 7 Chea...,"MCUILLA (Non-Privileged).pst\n\nHi Folks,\n\nA...",False,0.157376
45510,Message-ID: <29528344.1075857924287.JavaMail.e...,enron.announcements@enron.com,Remote control your air-conditioner and heater,lcampbel.nsf\n\nNewPower to offer another pilo...,False,0.172520
...,...,...,...,...,...,...
311648,Message-ID: <14061033.1075845796018.JavaMail.e...,kay.mann@enron.com,GE ContractPayment Summary,"kmann.nsf\n\nThe latest, greatest...\n--------...",False,0.105826
231068,Message-ID: <30988171.1075847650719.JavaMail.e...,mark.schroeder@enron.com,Benefits of competition in electricity\nCc: jo...,skean.nsf\n\nAs we all deal with the fallout f...,False,0.161730
257975,Message-ID: <8668663.1075840848630.JavaMail.ev...,laura.luce@enron.com,NY Event w/PEC & Keyspan\nCc: tammie.schoppe@e...,"louise kitchen 2-7-02.pst\n\nLouise,\nIt is my...",False,0.124851
478807,Message-ID: <15334805.1075859980268.JavaMail.e...,fishkinc@hotmail.com,,mtaylor.nsf\n\nHere's a working draft on a new...,False,0.168301
